In [4]:
#Code based on:
#https://www.r-bloggers.com/r-vs-python-image-classification-with-keras/
#Thanks!!


from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import ( Conv2D, Dense, LeakyReLU, BatchNormalization, MaxPooling2D, Dropout, Flatten)
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, TensorBoard
import PIL.Image
import pandas as pd
import numpy as np
from datetime import datetime as dt


In [5]:
start = dt.now()


#image categories
image_list = ["Rowing", "Erg", "Other"]

#number of output classes
output_n = len(image_list)

#image size to scale down to
img_width = 20
img_height = 20
target_size = (img_width, img_height)

#image rgb channel number
channels = 3

#path
path = "data/"
train_image_files_path = path + "Training"
valid_image_files_path = path + "Test"

In [6]:
## input data augmentation/modification
# training images
train_data_gen = ImageDataGenerator(rescale = 1./255)
# validation images
valid_data_gen = ImageDataGenerator(rescale = 1./255)

In [7]:
## get the data
# training images
train_image_array_gen = train_data_gen.flow_from_directory(train_image_files_path, target_size = target_size,
                                                           classes = image_list, class_mode = 'categorical',
                                                           seed = 42)
# validation images
valid_image_array_gen = valid_data_gen.flow_from_directory(valid_image_files_path, target_size = target_size,
                                                           classes = image_list, class_mode = 'categorical',
                                                           seed = 42)

Found 138 images belonging to 3 classes.
Found 27 images belonging to 3 classes.


In [8]:
## model definition
# number of training samples
train_samples = train_image_array_gen.n

# number of validation samples
valid_samples = valid_image_array_gen.n

# define batch size and number of epochs
batch_size = 32
epochs = 20

In [9]:
# initialise model
model = Sequential()

In [10]:
# add layers
# input layer
model.add(Conv2D(filters = 32, kernel_size = (3,3), padding = 'same', input_shape = (img_width, img_height, channels), activation = 'relu'))
# hiddel conv layer
model.add(Conv2D(filters = 16, kernel_size = (3,3), padding = 'same'))
model.add(LeakyReLU(.5))
model.add(BatchNormalization())
# using max pooling
model.add(MaxPooling2D(pool_size = (2,2)))
# randomly switch off 25% of the nodes per epoch step to avoid overfitting
model.add(Dropout(0.25))
# flatten max filtered output into feature vector
model.add(Flatten())
# output features onto a dense layer
model.add(Dense(units = 100, activation = 'relu'))
# randomly switch off 25% of the nodes per epoch step to avoid overfitting
model.add(Dropout(.5))
# output layer with the number of units equal to the number of categories
model.add(Dense(units = output_n, activation = 'softmax'))

In [11]:
# compile the model
model.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'],  optimizer = RMSprop(lr = 1e-4, decay = 1e-6))

In [12]:
callback_1 = [
    # save best model after every epoch
    ModelCheckpoint("rowing_checkpoints.h5", save_best_only = True),
    # only needed for visualising with TensorBoard
    TensorBoard(log_dir = "rowing_logs")
  ]

# train the model
hist = model.fit_generator(
  # training data
  train_image_array_gen,

  # epochs
  steps_per_epoch = train_samples // batch_size, 
  epochs = epochs, 

  # validation data
  validation_data = valid_image_array_gen,
  #validation_steps = valid_samples // batch_size,

  # print progress
  verbose = 2,
  callbacks=callback_1
)



Epoch 1/20
 - 13s - loss: 1.8906 - acc: 0.3667 - val_loss: 1.9154 - val_acc: 0.1481
Epoch 2/20
 - 15s - loss: 1.0839 - acc: 0.6406 - val_loss: 2.1088 - val_acc: 0.1111
Epoch 3/20
 - 11s - loss: 1.0441 - acc: 0.5651 - val_loss: 2.2366 - val_acc: 0.0741
Epoch 4/20
 - 11s - loss: 0.6763 - acc: 0.7040 - val_loss: 2.5080 - val_acc: 0.0741
Epoch 5/20
 - 11s - loss: 0.6716 - acc: 0.7421 - val_loss: 2.3674 - val_acc: 0.0741
Epoch 6/20
 - 11s - loss: 0.7831 - acc: 0.7315 - val_loss: 2.2129 - val_acc: 0.1111
Epoch 7/20
 - 11s - loss: 1.0138 - acc: 0.6027 - val_loss: 1.9805 - val_acc: 0.1111
Epoch 8/20
 - 11s - loss: 0.5171 - acc: 0.7497 - val_loss: 1.9784 - val_acc: 0.0741
Epoch 9/20
 - 11s - loss: 0.9695 - acc: 0.6571 - val_loss: 1.6114 - val_acc: 0.1852
Epoch 10/20
 - 14s - loss: 0.5207 - acc: 0.8125 - val_loss: 1.8084 - val_acc: 0.1852
Epoch 11/20
 - 11s - loss: 0.6361 - acc: 0.7584 - val_loss: 1.8232 - val_acc: 0.1481
Epoch 12/20
 - 12s - loss: 0.6378 - acc: 0.7284 - val_loss: 1.9770 - val_a

In [13]:
df_out = {'acc': hist.history['acc'][epochs - 1], 'val_acc': hist.history['val_acc'][epochs - 1], 'elapsed_time': (dt.now() - start).seconds}

In [14]:
print(df_out)


{'acc': 0.859375, 'val_acc': 0.37037035822868347, 'elapsed_time': 253}


In [15]:
# load_model_sample.py
from keras.models import load_model
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import os


def load_image(img_path, show=False):

    img = image.load_img(img_path, target_size=(20, 20))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor

In [17]:
def classify (Path):
# image path
    img_path = Path#'Data/Test/Rowing/IMG_0088.jpg'    # rowing
#img_path = '/media/data/dogscats/test1/19.jpg'      # erg

# load a single image
    new_image = load_image(img_path)

# check prediction
    pred = model.predict(new_image)

    print(pred)

In [27]:
classify('Data/Test/Rowing/IMG_0088.jpg' )
classify('Data/Test/Erg/IMG_0073.jpg' )
classify('Data/Test/Other/IMG_0015.jpg' )
classify('Data/Test/Other/simpsons.jpg' )

[[0.69822776 0.18845087 0.11332136]]
[[0.02419115 0.96180993 0.01399886]]
[[0.3906852  0.28799596 0.32131884]]
[[0.7196516  0.24432665 0.03602175]]


In [24]:

for item in image_list:
    
    path = 'Data/Test/'+item    
    #image_list    

    listing = os.listdir(path)    
    for file in listing:
        path = path+'/'+file
        if '.DS_Store' in file:
            continue
        classify(path)
       
    

NotADirectoryError: [Errno 20] Not a directory: 'Data/Test/Rowing/.DS_Store/IMG_0088.JPG'